# Advent of Code 2022

## Day 14: Regolith Reservoir

Solution code by [leechristie](https://github.com/leechristie) for Advent of Code 2022.

For part 1 I used a Numpy array to represent the viewable area, after sand tries to write somewhere with an `IndexError` raised, it's over and we return the number of grains which landed. I was not happy with this design even though it works.

After part 1 I stripped out the Numpy array from my code. Instead I use a `Cave` class which stores the state of the cave. Initially this is a wrapper for a Numpy array, then I switched out the backing data structure to a spare `dict` of points.

There's a flag in the construction of the data structure which treats the bottom as `ABYSS` or `FLOOR`. The `solve` method will check for either falling into the `ABYSS` or blocking the `START` location with `SAND`.

Part 2 new just calls the same solve method with the flag set.

The `show` method on `Cave` can be used to print the state of the cave, though this was not set in the final code it can be toggled back on by setting `show=True` in the call to `solve`.

In [ ]:
from typing import Iterator, Optional

In [ ]:
ABYSS = 'ABYSS'
FLOOR = 'FLOOR'
AIR = 'AIR'
ROCK = 'ROCK'
SAND = 'SAND'

NUM_HEADER_DIGITS = 3

### Reading Input

In [ ]:
def read_lines(filename: str) -> Iterator[list[tuple[int, int]]]:
    with open(filename) as file:
        for line in file:
            line = line.strip()
            line = line.split(' -> ')
            line = [tuple(e.split(',')) for e in line]
            line = [(int(x), int(y)) for x, y in line]
            yield line

In [ ]:
def read_all_points(filename: str) -> tuple[set[int], set[int]]:
    xs = set()
    ys = set()
    for line in read_lines(filename):
        for x, y in line:
            xs.add(x)
            ys.add(y)
    return ys, xs

### Looping Over The Rocks

In [ ]:
def is_single_point(from_point: tuple[int, int], to_point: tuple[int, int]) -> bool:
    return from_point == to_point

def is_vertical(from_point: tuple[int, int], to_point: tuple[int, int]) -> bool:
    return from_point[1] == to_point[1] and not from_point[0] == to_point[0]

def is_horizontal(from_point: tuple[int, int], to_point: tuple[int, int]) -> bool:
    return from_point[0] == to_point[0] and not from_point[1] == to_point[1]

def is_orthogonal(from_point: tuple[int, int], to_point: tuple[int, int]):
    return is_single_point(from_point, to_point) \
           or is_vertical(from_point, to_point) \
           or is_horizontal(from_point, to_point)

# yields all points between a start point and an end point
def path(from_point: tuple[int, int], to_point: tuple[int, int]) -> Iterator[tuple[int, int]]:
    assert is_orthogonal(from_point, to_point)
    if is_single_point(from_point, to_point):
        yield from_point
    elif is_horizontal(from_point, to_point):
        lower = min([from_point[1], to_point[1]])
        bound = max([from_point[1], to_point[1]]) + 1
        for x in range(lower, bound):
            yield from_point[0], x
    elif is_vertical(from_point, to_point):
        lower = min([from_point[0], to_point[0]])
        bound = max([from_point[0], to_point[0]]) + 1
        for y in range(lower, bound):
            yield y, from_point[1]
    else:
        raise AssertionError

# yields all rock points in a data file
def all_rocks(filename: str) -> Iterator[tuple[int, int]]:
    for line in read_lines(filename):
        previous = None
        for x, y in line:
            current = y, x
            if previous is not None:
                for point in path(previous, current):
                    yield point
            previous = current

### Data Structure

In [ ]:
class Cave:

    __slots__ = ['data', 'floor_height', 'floor_type']

    def __init__(self, floor_height: int, floor_type: str):
        self.data = {}
        self.floor_height = floor_height
        self.floor_type = floor_type

    def __getitem__(self, key: tuple[int, int]) -> str:
        if key in self.data:
            return self.data[key]
        y, _ = key
        if y >= self.floor_height:
            return self.floor_type
        return AIR

    def __setitem__(self, key: tuple[int, int], value: str) -> None:

        if value not in (ABYSS, FLOOR, AIR, ROCK, SAND):
            raise ValueError(f'unknown object type: {value}')

        # no action needed
        if self[key] == value:
            return

        # prevent accidentally overwriting parts of the cave
        if self[key] != AIR:
            raise IndexError(f'can not overwrite {self[key]} with {value}')

        self.data[key] = value

    @staticmethod
    def read_cave(filename: str, floor: bool=False) -> 'Cave':
        ys, xs = read_all_points(filename)
        rv = Cave(floor_height=max(ys)+2, floor_type=FLOOR if floor else ABYSS)
        for y, x in all_rocks(filename):
            rv[y, x] = ROCK
        return rv

    def calc_bounds(self) -> tuple[int, int, int, int]:

        left, right, top, bottom = None, None, None, None

        for y, x in self.data:
            if top is None or y < top:
                top = y
            if bottom is None or y > bottom:
                bottom = y
            if left is None or left > x:
                left = x
            if right is None or right < x:
                right = x

        # make sure the floor is shown
        if bottom < self.floor_height:
            bottom = self.floor_height

        return left, right, top, bottom

    def show(self, start: tuple[int, int]) -> None:

        left, right, top, bottom = self.calc_bounds()

        # print header row
        for place in range(NUM_HEADER_DIGITS):
            print('    ', end='')
            for x in range(left, right + 1):
                if x in (left, start[1], right):
                    print(str(x)[place], end='')
                else:
                    print(' ', end='')
            print()

        # print body rows
        for y in range(top, bottom+ 1):
            print(f'{y:>3} ', end='')
            for x in range(left, right + 1):
                if self[y, x] == AIR:
                    print('.', end='')
                elif self[y, x] == ROCK:
                    print('#', end='')
                elif self[y, x] == SAND:
                    print('o', end='')
                elif self[y, x] == ABYSS:
                    print('.', end='')
                elif self[y, x] == FLOOR:
                    print('#', end='')
                elif (y, x) == start:
                    print('+', end='')
                else:
                    raise AssertionError('unknown object for rendering at {y, x}')
            print()

### Sand Movement

In [ ]:
def step(current: tuple[int, int], state: Cave) -> tuple[Optional[tuple[int, int]], bool, bool]:

    # current point
    y, x = current
    assert (state[y, x] == AIR), f'current position of sand is {y, x} which is on top of {state[y, x]}'

    # directly down
    y, x = current
    y += 1
    if state[y, x] == AIR:
        return (y, x), True, False
    elif state[y, x] == ABYSS:
        return None, False, True

    # down and to the left
    y, x = current
    y += 1
    x -= 1
    if state[y, x] == AIR:
        return (y, x), True, False
    elif state[y, x] == ABYSS:
        return None, False, True

    # down and to the right
    y, x = current
    y += 1
    x += 1
    if state[y, x] == AIR:
        return (y, x), True, False
    elif state[y, x] == ABYSS:
        return None, False, True

    # come to rest
    return current, False, False

In [ ]:
def solve(state: Cave, start: tuple[int, int], show=False) -> int:
    sand_at_rest = 0
    sand, moving, ended = None, None, False
    while not ended:
        sand, moving, ended = start, True, ended
        if state[sand] == SAND:
            break
        while moving:
            sand, moving, ended = step(sand, state)
        if sand:
            assert state[sand] == AIR
            state[sand] = SAND
        if not ended:
            sand_at_rest += 1
    if show:
        state.show(start)
    return sand_at_rest

### Part 1

In [ ]:
INPUT_FILE = 'data/input14.txt'

In [ ]:
def main():
    state = Cave.read_cave(INPUT_FILE)
    sand_at_rest = solve(state, start=(0, 500), show=False)
    print(f'The number of grains of sand at rest is {sand_at_rest}')

In [ ]:
if __name__ == '__main__':
    main()

### Part 2

In [ ]:
def main():
    state = Cave.read_cave(INPUT_FILE, floor=True)
    sand_at_rest = solve(state, start=(0, 500), show=False)
    print(f'The number of grains of sand at rest is {sand_at_rest}')

In [ ]:
if __name__ == '__main__':
    main()